In [1]:
import pandas as pd
import duckdb

In [2]:
duckdb.sql('SET enable_progress_bar = true')

In [3]:
# Load the data
duckdb.sql('''
           select
             *
           from '../../data/taxi_trips.csv'
           limit 10
           '''
).df().T

,0,1,2,3,4,5,6,7,8,9
VendorID,2,2,2,2,2,1,1,2,2,2
tpep_pickup_datetime,2018-07-17 10:14:57,2018-07-17 10:23:50,2018-07-17 10:39:57,2018-07-17 10:12:58,2018-07-17 10:24:49,2018-07-17 10:29:59,2018-07-17 10:44:55,2018-07-17 10:02:58,2018-07-17 10:23:14,2018-07-17 10:01:52
tpep_dropoff_datetime,2018-07-17 10:22:45,2018-07-17 10:37:49,2018-07-17 11:07:52,2018-07-17 10:22:47,2018-07-17 10:48:37,2018-07-17 10:41:50,2018-07-17 10:59:11,2018-07-17 10:31:42,2018-07-17 10:50:12,2018-07-17 10:06:29
passenger_count,2,2,2,1,1,1,1,1,5,1
trip_distance,1.0,1.24,5.12,0.88,2.07,1.2,1.3,4.03,2.31,0.63
RatecodeID,1,1,1,1,1,1,1,1,1,1
store_and_fwd_flag,N,N,N,N,N,N,N,N,N,N
PULocationID,186,234,186,48,186,50,48,246,237,230
DOLocationID,234,186,88,186,163,100,186,209,233,163
payment_type,1,1,1,2,1,1,2,1,1,1


In [4]:
def qw(query):
    """
    Helper function to execute a query and return the result as a DataFrame.
    """
    with duckdb.connect('../../taxi_data.duckdb') as con:
        return con.execute(query).fetchdf()
    with duckdb.connect('../../taxi_data.duckdb') as con:
        df = con.execute(query).fetchdf()

    return df

In [5]:
qw('''
SELECT * FROM dbt_models.mart_trip_summary
''')

,summary_type,total_trips
0,daily_summary,111411378


In [6]:
qw('''
SELECT * FROM dbt_models.payment_types
''')

,id,name
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown
5,6,Voided trip


In [7]:
qw('''
SELECT * FROM dbt_models.payment_types
''')

,id,name
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown
5,6,Voided trip


In [8]:
qw('''
SELECT *
FROM dbt_models.ratecodes
''')

,id,name
0,1,Standard rate
1,2,JFK
2,3,Newark
3,4,Nassau or Westchester
4,5,Negotiated fare
5,6,Group ride


In [19]:
qw('SELECT * FROM information_schema.tables')

,0,1,2,3,4
table_catalog,taxi_data,taxi_data,taxi_data,taxi_data,taxi_data
table_schema,dbt_models,dbt_models,dbt_models,main,dbt_models
table_name,mart_trip_summary,payment_types,ratecodes,taxi_trips,stg_taxi_trips
table_type,BASE TABLE,BASE TABLE,BASE TABLE,BASE TABLE,VIEW
self_referencing_column_name,None,None,None,None,None
reference_generation,None,None,None,None,None
user_defined_type_catalog,None,None,None,None,None
user_defined_type_schema,None,None,None,None,None
user_defined_type_name,None,None,None,None,None
is_insertable_into,YES,YES,YES,YES,NO


In [10]:
qw('''
   SELECT
     taxi_trips.pickup_datetime,
     taxi_trips.trip_distance,
     ratecodes.name AS rate_code_name,
     payment_types.name AS payment_type_name
   FROM dbt_models.stg_taxi_trips AS taxi_trips
   LEFT JOIN dbt_models.ratecodes AS ratecodes
          ON taxi_trips.rate_code_id = ratecodes.id
   LEFT JOIN dbt_models.payment_types AS payment_types
          ON taxi_trips.payment_type = payment_types.id
   LIMIT 10
   ''')

,pickup_datetime,trip_distance,rate_code_name,payment_type_name
0,2018-07-18 08:37:42,2.40,Standard rate,Credit card
1,2018-07-18 08:56:21,0.70,Standard rate,Credit card
2,2018-07-18 08:03:41,0.92,Standard rate,Credit card
3,2018-07-18 08:14:54,2.98,Standard rate,Credit card
4,2018-07-18 08:42:46,0.40,Standard rate,Cash
5,2018-07-18 08:45:46,1.89,Standard rate,Credit card
6,2018-07-18 08:03:08,1.84,Standard rate,Credit card
7,2018-07-18 08:13:05,0.68,Standard rate,Cash
8,2018-07-18 08:22:08,1.16,Standard rate,Credit card
9,2018-07-18 08:41:50,2.17,Standard rate,Credit card


In [16]:
qw('''
   SELECT
     taxi_trips.rate_code_id,
     ratecodes.name AS rate_code_name,
     count(*)
   FROM dbt_models.stg_taxi_trips AS taxi_trips
   LEFT JOIN dbt_models.ratecodes AS ratecodes
          ON taxi_trips.rate_code_id = ratecodes.id
   GROUP BY 1,2
   ORDER BY 1,2
   ''')

,rate_code_id,rate_code_name,count_star()
0,1,Standard rate,108895988
1,2,JFK,2589646
2,3,Newark,228859
3,4,Nassau or Westchester,70115
4,5,Negotiated fare,443848
5,6,Group ride,948
6,<NA>,None,5222


In [21]:
qw('''
   select *
   from dbt_models.stg_taxi_trips as taxi_trips
   where pickup_datetime > dropoff_datetime
   limit 100
   ''').T

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

""
vendor_id
pickup_datetime
dropoff_datetime
passenger_count
trip_distance
rate_code_id
store_and_fwd_flag
pickup_location_id
dropoff_location_id
payment_type


In [22]:
qw('describe dbt_models.stg_taxi_trips')

,column_name,column_type,null,key,default,extra
0,vendor_id,INTEGER,YES,None,None,None
1,pickup_datetime,TIMESTAMP,YES,None,None,None
2,dropoff_datetime,TIMESTAMP,YES,None,None,None
3,passenger_count,INTEGER,YES,None,None,None
4,trip_distance,"DECIMAL(10,2)",YES,None,None,None
5,rate_code_id,INTEGER,YES,None,None,None
6,store_and_fwd_flag,VARCHAR,YES,None,None,None
7,pickup_location_id,INTEGER,YES,None,None,None
8,dropoff_location_id,INTEGER,YES,None,None,None
9,payment_type,INTEGER,YES,None,None,None


In [29]:
qw('''
SELECT * FROM dbt_models.mart_distance_distribution
   ''')

,total_trips,avg_trip_distance_km,stddev_trip_distance_km,min_trip_distance_km,max_trip_distance_km,q1_trip_distance_km,median_trip_distance_km,q3_trip_distance_km
0,112233579,2.929427,18.337476,0.0,189483.84,1,2,3


In [30]:
qw('''
SELECT * FROM dbt_models.mart_distance_distribution_km
   ''')

,total_trips,avg_trip_distance_km,stddev_trip_distance_km,min_trip_distance_km,max_trip_distance_km,q1_trip_distance_km,median_trip_distance_km,q3_trip_distance_km
0,112233579,4.714416,29.511235,0.0,304943.92,2,3,5
